In [4]:
from re import DEBUG
from flask import Flask, jsonify, request ,render_template, send_file
import pickle
from PIL import Image, ImageDraw
from torchvision.transforms import functional as func
import torch
import base64
import io
import re
import json
from base64 import decodestring
import numpy as np
import cv2

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

font = cv2.FONT_HERSHEY_SIMPLEX

app=Flask(__name__)


@app.route('/predictObject',methods=["POST"])
def predict():
    imagestr=""
    
    data=None
    content_type = request.headers.get('Content-Type')
    if (content_type == 'application/json; charset=utf-8'):
        print("running1")
        data = json.loads(request.data.decode("utf-8") )
        print(data)
        print("running2")

    #     print(data)
        imagestr = data['imagefile']
    # process Image
        print("Image received")
        imagestr = re.sub('^data:image/.+;base64,', '', imagestr)
        decoded = base64.b64decode(imagestr)
        
#         img = Image.open(io.BytesIO(decoded))
        img=cv2.imread(str(io.BytesIO(decoded)))
        # Run the model on GPU if it is available
        img =cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        image = func.to_tensor(img)
        # image=torch.tensor(image)
        c, h, w = image.shape
    #     img.show()
        # Perform inference
        preds = model([image])[0]
        labels = preds["labels"].cpu().detach().numpy()
        scores = preds["scores"].cpu().detach().numpy()
        boxes = preds["boxes"].cpu().detach().numpy()
 
        result={}
        for label, score, box in zip(labels, scores, boxes):
            # Convert to [top-left-x, top-left-y, width, height]
            # in relative coordinates in [0, 1] x [0, 1]
            if score>= .9 :
                x1, y1, x2, y2 = box
    #             rel_box = [x1 / w, y1 / h, (x2r - x1) / w, (y2 - y1) / h]
                rel_box=[x1, y1, x2, y2]
                name=label if len(COCO_INSTANCE_CATEGORY_NAMES)<label else COCO_INSTANCE_CATEGORY_NAMES[label]
                #result[label]=[name,score]
#                 img1.rectangle(rel_box,  outline ="blue")
                cv2.rectangle(img,(x1,y1),(x2,y3),(0,0,255),15)
                cv2.putText(img,"{} : {:.2f}".format(name,score),(10,500), font, 6, (200,255,155), 13, cv2.LINE_AA)
                
        imgByteArr = io.BytesIO()
        img = np.array(img, dtype=np.uint8)
        img=Image.fromarray(img)
        img.save(imgByteArr, format='jpeg')
        imgByteArr = imgByteArr.getvalue()
        print("Image sending")
        return str(jsonify({"resultant_image" : str(base64.b64encode(imgByteArr))}))
    
        
    else:
        print('Content-Type not supported!'+content_type)
        return 'Content-Type not supported!'

# http://10.0.2.2:8080/
if __name__ == '__main__':
    with open('mlmodel.pickle', 'rb') as f:
            model = pickle.load(f)
    app.run(port=2000,debug=True, use_reloader=False)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:2000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jun/2022 14:46:42] "POST /predictObject HTTP/1.1" 500 -


running1
{'imagefile': '...'}
running2
Image received


Traceback (most recent call last):
  File "C:\Users\athar\anaconda3\Lib\site-packages\flask\app.py", line 2091, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\athar\anaconda3\Lib\site-packages\flask\app.py", line 2076, in wsgi_app
    response = self.handle_exception(e)
  File "C:\Users\athar\anaconda3\Lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\athar\anaconda3\Lib\site-packages\flask\app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\athar\anaconda3\Lib\site-packages\flask\app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\athar\anaconda3\Lib\site-packages\flask\app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\athar\AppData\Local\Temp\ipykernel_22384\1878043410.py", line 57, in predict
    
cv2.error: 